In [1]:
import sys
sys.path.insert(0,"/mnt/c/githubreps/SlowQuant/")
import numpy as np
from slowquant import shortcuts as SQ

In [52]:
pathtobasis = '/mnt/c/githubreps/SlowQuant/slowquant/basissets/'
settings    = {'basisset':'6-31G'}
molecule    = np.array([[4,0,0,0],
                        [3,3.014112952,0.0,0.0],
                        [1,0.0,0.0,0.0]])

In [53]:
VNN, VNe, S, Te, Vee = SQ.Integrals(molecule, settings, pathtobasis)

In [75]:
EHF, C, F, D, iterations = SQ.HartreeFock(molecule, VNN, Te, S, VNe, Vee, do_diis='Yes')

In [76]:
VeeMO = SQ.Transform2eMO(Vee, C)
VeeMOspin = SQ.Transform2eSPIN(Vee)

In [77]:
# Convert everything needed to spin, to match equations of http://sci-hub.cc/10.1016/0009-2614(88)80249-5
Fspin = np.zeros((len(F)*2,len(F)*2))
Cspin = np.zeros((len(F)*2,len(F)*2))
for p in range(1,len(F)*2+1):
    for q in range(1,len(F)*2+1):
        Fspin[p-1,q-1] = F[(p+1)//2-1,(q+1)//2-1] * (p%2 == q%2)
        Cspin[p-1,q-1] = C[(p+1)//2-1,(q+1)//2-1] * (p%2 == q%2)
FMOspin = np.dot(np.transpose(Cspin),np.dot(Fspin,Cspin))
occ = int(molecule[0,0])
dimension = len(FMOspin)
def doublebar(i,j,k,l, Vee):
    return Vee[i,j,k,l] - Vee[i,j,l,k]

In [78]:
# Constructing the t amplitudes
tdouble = np.zeros((dimension,dimension,dimension,dimension))
for i in range(0, occ):
    for j in range(0, occ):
        for a in range(occ, dimension):
            for b in range(occ, dimension):
                Dijab = FMOspin[i,i]+FMOspin[j,j]-FMOspin[a,a]-FMOspin[b,b]
                tdouble[i,j,a,b] = doublebar(i,j,a,b,VeeMOspin)/Dijab

In [88]:
DMP2 = np.zeros((dimension,dimension))

# Constructing the MP2 density
for i in range(0, occ):
    for j in range(0, occ):
        for k in range(0, occ):
            for a in range(occ, dimension):
                for b in range(occ, dimension):
                    DMP2[i,j] += -0.5*tdouble[j,k,a,b]*tdouble[i,k,a,b]

for a in range(occ, dimension):
    for b in range(occ, dimension):
        for i in range(0, occ):
            for j in range(0, occ):
                for c in range(occ, dimension):
                    DMP2[a,b] += 0.5*tdouble[i,j,a,c]*tdouble[i,j,b,c]

# Constructing the total density
Dtot = np.zeros((dimension,dimension))
for i in range(0, dimension):
    for j in range(0, dimension):
        if i == j and i < occ and j < occ:
            Dtot[i,j] = 1 + DMP2[i,j]
        else:
            Dtot[i,j] = DMP2[i,j]

In [89]:
from numpy.linalg import eig
# Determine the natural orbitals
NO, CNO = eig(Dtot)

In [92]:
# It have to sum to four
NO, np.sum(NO)

(array([  9.99836479e-01 +0.00000000e+00j,
          8.24658677e-01 +0.00000000e+00j,
          9.99836479e-01 +0.00000000e+00j,
          8.24658677e-01 +0.00000000e+00j,
          1.43997569e-02 +0.00000000e+00j,
          1.41534448e-01 +0.00000000e+00j,
          1.41534448e-01 +0.00000000e+00j,
          1.43997569e-02 +0.00000000e+00j,
          4.48759865e-04 +0.00000000e+00j,
          4.48759865e-04 +0.00000000e+00j,
          1.09720969e-02 +0.00000000e+00j,
          1.09720969e-02 +0.00000000e+00j,
          9.18256367e-05 +0.00000000e+00j,
          9.18256367e-05 +0.00000000e+00j,
          8.04556026e-03 +0.00000000e+00j,
          8.04556026e-03 +0.00000000e+00j,
          4.85800862e-06 +0.00000000e+00j,
          4.85800862e-06 +0.00000000e+00j,
          4.39501442e-06 +2.47342413e-19j,
          4.39501442e-06 -2.47342413e-19j,
          3.14247096e-06 +0.00000000e+00j,   3.14247096e-06 +0.00000000e+00j]),
 (4+0j))

In [93]:
CNO

array([[  9.98135744e-01 +0.00000000e+00j,
          6.10330759e-02 +0.00000000e+00j,
          7.36791213e-02 +0.00000000e+00j,
          3.70823913e-03 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 -0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j,
          0.00000000e+00 +0.00000000e+00j],
       [  0.00000000e+00 +0.00000000e+00j,
          